In [57]:
from transformers import XLMTokenizer, RobertaModel
import glob
import pandas as pd
import pickle
import numpy as np
from tqdm.auto import tqdm, trange
import torch
from torch import nn
import random
import math
import matplotlib.pyplot as plt
import csv
from sentimentpl.models import SentimentPLModel

In [60]:
model = SentimentPLModel(from_pretrained='latest')
model.to('cuda')
pass

In [ ]:
count = 10

for filename in glob.glob('sentiment_data/#*.csv'): 
    with open(filename, 'r', encoding='utf-8') as f:
        f_len = len([1 for _ in f])
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t', quotechar='"')
        next(reader, None)
        res = [(row[10], model([row[10]]).item()) for row in tqdm(reader, total=f_len)]
        most_positive = sorted(res, key=lambda x: -x[1])[:count]
        most_negative = sorted(res, key=lambda x: x[1])[:count]
    
    print('\n\n', filename, '\n')
    print('\n\nmost positive')
    for e in most_positive:
        print(e[0])
        print(e[1])
        print()
    print('\n\nmost negative')
    for e in most_negative:
        print(e[0])
        print(e[1])
        print()

In [59]:
count = 20
output_filename = f'hashtag_sampled_{count}.csv'

output = []
for filename in tqdm(glob.glob('sentiment_data/#*.csv')): 
    with open(filename, 'r', encoding='utf-8') as f:
        lines = [l.split('\t') for l in f][1:]
    output += [f'#{filename.split("#")[-1].split(".csv")[0]},{row[0]},"{row[10]}",{model([row[10]]).item()}\n' 
              for row in random.sample(lines, min(count, len(lines)))]
with open(output_filename, 'w', encoding='utf-8') as f:
    f.writelines(output)